In [1]:
import torch
from torch.autograd import Variable
import numpy as np

In [2]:
xy = np.loadtxt('data/diabetes.csv', delimiter=',', dtype=np.float32)
#불러온 csv 파일에서 마지막 열이 정답 값을 가지고 있다.
x_data = torch.from_numpy(xy[:, 0:-1]) #759 x 8, 한번에 8개의 변수들이 있다.
y_data = torch.from_numpy(xy[:, [-1]]) #759 x 1, T/F 이므로 1개의 값만 있다.
#예측 값 #4.0부터는 Variable 없이 그냥 Tensor를 그대로 넣어도 된다.
#여러 개의 데이터를 동시에 입력할 경우, 행렬로 표현해 주면 된다. 한 레이어에서 여러 값들을 계산하므로 network가 넓어 진다.

print(x_data.data.shape)
print(y_data.data.shape)

torch.Size([759, 8])
torch.Size([759, 1])


## 1. Design Your model using class with Variables

In [28]:
class Model(torch.nn.Module): #Module상속 
    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(8, 6) #input_dim을 x_data의 size와 맞춰줘야 한다.
        self.l2 = torch.nn.Linear(6, 4)
        self.l3 = torch.nn.Linear(4, 1) #output_dim을 y_data의 size와 맞춰줘야 한다.
        #layer 여러 개를 input dimention과 output dimension을 맞춰 연결 해 network를 더 깊게 만들어 줄 수 있다.
        
        self.sigmoid = torch.nn.Sigmoid() #Sigmoid는 값을 0 ~ 1 사이의 값으로 변환시켜 주므로, 확률로 T/F를 판단할 수 있다.
        #Logistic regression은 Linear regression에 로지스틱 함수(ex. Sigmoid)를 추가하고, 
        #손실함수를 변경해(ex. Cross Entropy) 구현할 수 있다.
        #4.0에서 torch.nn.functional.sigmoid 가 torch.sigmoid 로 변경되었다.
        
    def forward(self, x): #train, predict. #nn.Module class 에서 반드시 구현해야 한다.
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        out1 = self.sigmoid(self.l1(x)) #l1의 결과값을 sigmoid
        out2 = self.sigmoid(self.l2(out1)) #l2의 결과값을 sigmoid
        y_pred = self.sigmoid(self.l3(out2)) #l3의 결과값을 sigmoid
        #Sigmoid를 활성함수로 잘 쓰지 않는 이유는 Vanishing Gradient Problem이 생기기 때문이다. 
        #Sigmoid를 사용하면 미분값이 0과 1사이가 되는데, 이를 역전파하면서 계속 곱해주면 0에 근접한 값이 되므로 제대로 업데이트를 할 수 없다.
        return y_pred
    
    #Linear regression은 output이 값, Logistic regression은 output이 T/F(1/0)
    
model = Model() # model 생성

## 2. Construct loss and optimizer(Select from PyTorch API)

In [29]:
# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.BCELoss(reduction='elementwise_mean') #손실함수 Binary Cross Entropy
#Logistic regression은 Linear regression에 로지스틱 함수(ex. Sigmoid)를 추가하고, 
#손실함수를 변경해(ex. Cross Entropy) 구현할 수 있다.
optimizer = torch.optim.SGD(model.parameters(), lr=0.1) #최적화함수
#model.parameters() 로 업데이트해야 할 모든 변수들을 한 번에 가져와 간단히 구현할 수 있다.
#손실에서 lr만큼 움직이고 업데이트 하던 것을 알아서 최적화해서 처리해 준다.

## 3. Training Cycle(Foward, Backward, Update)

In [30]:
# Training loop
for epoch in range(100): #100번 반복해서 training
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x_data) #예측 값 #4.0부터는 Variable 없이 그냥 Tensor를 그대로 넣어도 된다.

    # Compute and print loss
    loss = criterion(y_pred, y_data) #예측한 값과, 정답
    print(epoch, loss.item()) #loss.data[0]로 출력하던 것을 loss.item()으로 변경되었다.

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad() #optimiser.step() 으로 업데이트된 그라디언트 값들을 초기화해 줘야 한다.
    loss.backward() #역전파 해 준다. 각 변수의 기울기를 구한다.
    optimizer.step() #변수 업데이트

0 0.6776887774467468
1 0.674457848072052
2 0.6715431213378906
3 0.668914794921875
4 0.6665436625480652
5 0.6644051671028137
6 0.6624758243560791
7 0.6607356071472168
8 0.6591653823852539
9 0.6577489972114563
10 0.6564704775810242
11 0.6553173661231995
12 0.6542761921882629
13 0.6533360481262207
14 0.6524883508682251
15 0.6517223715782166
16 0.6510305404663086
17 0.650406002998352
18 0.6498421430587769
19 0.6493323445320129
20 0.6488722562789917
21 0.648456335067749
22 0.6480802893638611
23 0.6477404236793518
24 0.6474336981773376
25 0.6471557021141052
26 0.6469048857688904
27 0.6466780304908752
28 0.646472692489624
29 0.6462870836257935
30 0.6461187601089478
31 0.6459670066833496
32 0.6458293795585632
33 0.6457050442695618
34 0.6455925703048706
35 0.6454904675483704
36 0.6453983783721924
37 0.6453142166137695
38 0.6452383995056152
39 0.6451699733734131
40 0.6451078057289124
41 0.6450519561767578
42 0.6450008153915405
43 0.6449543833732605
44 0.6449121236801147
45 0.644873857498169
46 0

## Customized Model

In [10]:
from collections import OrderedDict
#정확도를 향상 시키는 방법에는

class Model(torch.nn.Module): 
    def __init__(self, input_num=8, output_num=1):
        super(Model, self).__init__()
        
        self.model = torch.nn.Sequential(OrderedDict([ #레이어를 깊게 하고
            ("fc1", torch.nn.Linear(input_num, 20)),
            ("sig1", torch.nn.Sigmoid()),
            ("fc2", torch.nn.Linear(20, 40)),
            ("sig2", torch.nn.Sigmoid()),
            ("fc3", torch.nn.Linear(40, 80)),
            ("sig3", torch.nn.Sigmoid()),
            ("fc4", torch.nn.Linear(80, 40)),
            ("sig4", torch.nn.Sigmoid()),
            ("fc5", torch.nn.Linear(40, 20)),
            ("sig5", torch.nn.Sigmoid()),
            ("fc6", torch.nn.Linear(20, 10)),
            ("sig6", torch.nn.Sigmoid()),
            ("fc7", torch.nn.Linear(10, output_num)),
            ("sig7", torch.nn.Sigmoid()),
        ]))
        
    def forward(self, x): 
        return self.model(x)
    
model = Model()

criterion = torch.nn.BCELoss(reduction='elementwise_mean')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #최적화 함수를 바꾸고, lr을 조정


# Training loop
for epoch in range(1000): #epoch을 늘려준다.
    y_pred = model(x_data)
    
    # Compute and print loss
    loss = criterion(y_pred, y_data) 
    print(epoch, loss.item()) 

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad() 
    loss.backward()
    optimizer.step()
    
#이 외에도 활성함수 변경, 손실함수 변경, pooling 등이 있다.

0 0.6946731209754944
1 0.6928844451904297
2 0.6911194920539856
3 0.6893888711929321
4 0.6876886487007141
5 0.6860215663909912
6 0.6843922138214111
7 0.682791531085968
8 0.6812154650688171
9 0.6796801090240479
10 0.6781781911849976
11 0.676713764667511
12 0.6752853989601135
13 0.6738844513893127
14 0.6725260019302368
15 0.6712067723274231
16 0.6699172258377075
17 0.6686628460884094
18 0.6674491167068481
19 0.666273295879364
20 0.6651301383972168
21 0.6640269756317139
22 0.6629586219787598
23 0.6619309186935425
24 0.6609395146369934
25 0.6599957942962646
26 0.6590763926506042
27 0.6582029461860657
28 0.6573613286018372
29 0.6565518975257874
30 0.6557837128639221
31 0.6550544500350952
32 0.6543586850166321
33 0.6536955237388611
34 0.6530641317367554
35 0.6524672508239746
36 0.6519044637680054
37 0.6513767838478088
38 0.650875985622406
39 0.650409460067749
40 0.6499722003936768
41 0.6495614051818848
42 0.6491772532463074
43 0.6488201022148132
44 0.6484857201576233
45 0.6481756567955017
46 

378 0.6447305083274841
379 0.6447052955627441
380 0.6446803212165833
381 0.6446526050567627
382 0.6446226835250854
383 0.6445924639701843
384 0.6445595026016235
385 0.6445245146751404
386 0.6444873213768005
387 0.6444472074508667
388 0.6444052457809448
389 0.6443600058555603
390 0.6443120241165161
391 0.6442603468894958
392 0.6442050933837891
393 0.6441457271575928
394 0.6440819501876831
395 0.6440136432647705
396 0.6439394950866699
397 0.6438606381416321
398 0.6437755823135376
399 0.6436834931373596
400 0.6435838937759399
401 0.6434764266014099
402 0.6433601379394531
403 0.6432344913482666
404 0.6430976986885071
405 0.6429510712623596
406 0.6427916884422302
407 0.6426186561584473
408 0.6424312591552734
409 0.6422287225723267
410 0.6420091390609741
411 0.6417717337608337
412 0.6415146589279175
413 0.6412367224693298
414 0.6409367322921753
415 0.6406122446060181
416 0.6402624249458313
417 0.6398854851722717
418 0.6394801139831543
419 0.6390434503555298
420 0.63857501745224
421 0.6380723

751 0.4529743790626526
752 0.452911376953125
753 0.4528493881225586
754 0.4527862071990967
755 0.45272400975227356
756 0.4526618421077728
757 0.45259949564933777
758 0.45253753662109375
759 0.45247599482536316
760 0.45241284370422363
761 0.4523516595363617
762 0.452289879322052
763 0.45222869515419006
764 0.4521666467189789
765 0.4521051347255707
766 0.4520440697669983
767 0.45198240876197815
768 0.4519212543964386
769 0.4518599808216095
770 0.45179855823516846
771 0.4517371952533722
772 0.45167654752731323
773 0.45161569118499756
774 0.45155444741249084
775 0.45149344205856323
776 0.4514330327510834
777 0.4513717591762543
778 0.4513107240200043
779 0.4512498676776886
780 0.4511888325214386
781 0.4511280357837677
782 0.4510669708251953
783 0.45100629329681396
784 0.45094558596611023
785 0.4508846700191498
786 0.45082420110702515
787 0.45076316595077515
788 0.45070257782936096
789 0.4506419003009796
790 0.4505808353424072
791 0.45051923394203186
792 0.4504588842391968
793 0.450397729873